In [1]:
##WithColumn example
from pyspark.sql import Row
from pyspark.sql.types import *
rdd=sc.parallelize([1,2,3,4])
rdd.collect()
rowRdd=rdd.map(lambda x: Row(name=x))
schema = StructType([StructField("name", IntegerType(), False)])
df=sqlContext.createDataFrame(data=rowRdd,schema=schema)
df2=df.select('name')
df3=df.withColumn('id',df.name+2)
df3.show()

+----+---+
|name| id|
+----+---+
|   1|  3|
|   2|  4|
|   3|  5|
|   4|  6|
+----+---+



In [2]:
datafrm=sc.parallelize([Row(id='a',value1=1,value2=2),Row(id='b',value1=2,value2=3),Row(id='a',value1=3,value2=4)]).toDF()
computedOne=datafrm.withColumn('newCol',datafrm.value1*datafrm.value2)
averagedDataFrame=datafrm.groupBy('id').avg('value1')
averagedDataFrame.show()
sqlContext.registerDataFrameAsTable(df=datafrm,tableName='values')
avgDF=sqlContext.sql('select avg(value1),id from values group by id')
avgDF.show()

+---+-----------+
| id|avg(value1)|
+---+-----------+
|  b|        2.0|
|  a|        2.0|
+---+-----------+

+-----------+---+
|avg(value1)| id|
+-----------+---+
|        2.0|  b|
|        2.0|  a|
+-----------+---+



In [4]:
##Calculating previous Date in dataFrame
from datetime import datetime,timedelta
from pyspark.sql.functions import when,date_add,col
dateDF=sc.parallelize([Row(name='a',dat=datetime(2018,4,20)),
                       Row(name='b',dat=datetime(2018,4,20)),
                       Row(name='a',dat=datetime(2018,4,20))]).toDF()
addedColumn=dateDF.withColumn('prevDate',when(dateDF.dat.isNotNull(),date_add(col('dat'),-1)).otherwise(dateDF['dat']))
addedColumn.show()

+-------------------+----+-------------------+
|                dat|name|           prevDate|
+-------------------+----+-------------------+
|2018-04-20 00:00:00|   a|2018-04-19 00:00:00|
|2018-04-20 00:00:00|   b|2018-04-19 00:00:00|
|2018-04-20 00:00:00|   a|2018-04-19 00:00:00|
+-------------------+----+-------------------+



In [5]:
from pyspark.sql import functions as F
##range to generate df
rangeDF=spark.range(1,10,1)
rangeDF.show()
##data frame select and where function
filteredDF=rangeDF.select('id').where(rangeDF.id>5)
filteredDF.show()
#agg function
maxId=filteredDF.agg(F.avg(filteredDF.id))
maxId.show()
##approxquantile
#quartDF=rangeDF.approxQuantile('id',[0,1],0.5)
#quartDF.show()
rangeDF.columns

##cross join
crossDF=rangeDF.crossJoin(filteredDF)
crossDF.show()
crossDF.describe().show()
crossDF.drop('id').show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

+---+
| id|
+---+
|  6|
|  7|
|  8|
|  9|
+---+

+-------+
|avg(id)|
+-------+
|    7.5|
+-------+

+---+---+
| id| id|
+---+---+
|  1|  6|
|  1|  7|
|  1|  8|
|  1|  9|
|  2|  6|
|  2|  7|
|  2|  8|
|  2|  9|
|  3|  6|
|  3|  7|
|  3|  8|
|  3|  9|
|  4|  6|
|  4|  7|
|  4|  8|
|  4|  9|
|  5|  6|
|  5|  7|
|  5|  8|
|  5|  9|
+---+---+
only showing top 20 rows

+-------+------------------+------------------+
|summary|                id|                id|
+-------+------------------+------------------+
|  count|                36|                36|
|   mean|               5.0|               7.5|
| stddev|2.6186146828319083|1.1338934190276817|
|    min|                 1|                 6|
|    max|                 9|                 9|
+-------+------------------+------------------+

++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++
only showing top 20 rows



In [8]:
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql import Row

def rowKeyFunction(x):
    return x.Salary

def ranking(x):
    y=[]
    for index,element in enumerate(x):
        dictEle=element.asDict()
        if(index==0):
            dictEle['rank']=1
        elif element.Salary==x[index-1].Salary:
            dictEle['rank']=y[index-1]['rank']
        else:
            dictEle['rank']=y[index-1]['rank']+1
        y.append(Row(**dictEle))
    return y
        
empDf=sqlContext.read.csv(path='/home/kapil/software-apps/input-files/employee-input.csv',header=True,inferSchema=True)
empDf.registerTempTable('employee')
##rdd approach
empRdd=empDf.rdd;
windowPartitonRdd=empRdd.groupBy(lambda x:x['EmpNo'])
sortedRdd=windowPartitonRdd.mapValues(lambda y:sorted(y,key=rowKeyFunction))
rankedRdd=sortedRdd.mapValues(ranking)
rowRdd=rankedRdd.values().flatMap(lambda x:x)
createdDf=rowRdd.toDF()
createdDf.show()


##by usig window function
windowSpec=Window.partitionBy(empDf['EmpNo']).orderBy(empDf['Salary'])
##with joining two dataframes
windowedDF=empDf.distinct().select(func.rank().over(windowSpec).alias('EmpRank'),empDf.EmpNo,empDf.Salary)
joinedDF=empDf.join(other=windowedDF,on=([windowedDF.EmpNo==empDf.EmpNo,windowedDF.Salary==empDf.Salary]),how='inner')
joinedDF.show()



## using with column function and dense rank
withColumnDf=empDf.withColumn(col=func.dense_rank().over(windowSpec),colName='rank')
withColumnDf.show()


##using sql
sqlDf=sqlContext.sql('select EmpNo,EmpName,Salary, dense_rank() OVER (PARTITION BY EmpNo ORDER BY Salary ASC) as rank from employee')
sqlDf.show() 



+-------+-----+------+----+
|EmpName|EmpNo|Salary|rank|
+-------+-----+------+----+
|    Som| E125|  4000|   1|
|    Som| E125|  6000|   2|
|    Tom| E123|  2000|   1|
|    Tom| E123|  2000|   1|
|    Tom| E123|  8000|   2|
|    Rom| E124|  3000|   1|
|    Rom| E124|  7000|   2|
|    Pom| E126|  5000|   1|
+-------+-----+------+----+

+-----+-------+------+-------+-----+------+
|EmpNo|EmpName|Salary|EmpRank|EmpNo|Salary|
+-----+-------+------+-------+-----+------+
| E123|    Tom|  2000|      1| E123|  2000|
| E124|    Rom|  3000|      1| E124|  3000|
| E125|    Som|  4000|      1| E125|  4000|
| E126|    Pom|  5000|      1| E126|  5000|
| E125|    Som|  6000|      2| E125|  6000|
| E124|    Rom|  7000|      2| E124|  7000|
| E123|    Tom|  8000|      2| E123|  8000|
| E123|    Tom|  2000|      1| E123|  2000|
+-----+-------+------+-------+-----+------+

+-----+-------+------+----+
|EmpNo|EmpName|Salary|rank|
+-----+-------+------+----+
| E126|    Pom|  5000|   1|
| E125|    Som|  4000|